Link to GDrive

In [5]:
# Mount GDrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# **ATTENTION** (cf ci-dessous)

Choisir entre

*   Bloc 1 : Génération d'un fichier csv "aléatoire" à partir du csv initial (479Mo)
*   Bloc 2 : On repart du dernier csv généré pour réexecuter le data cleaning

original (sans doublons)

In [13]:
import pandas as pd
import random

file_path = "/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/"


#
# CHOISIR entre Bloc 1 et Bloc 2 selon que l'on veut regénérer (Bloc 1) ou pas (Bloc 2) un fichier csv initial
# Enlever ou Mettre en commentaires selon le choix
# Bloc 1
#
## n = sum(1 for line in open(file)) - 1 #number of records in file (excludes header)
## s = 1000 #desired sample size
## skip = sorted(random.sample(range(1,n+1),n-s)) #the 0-indexed header will not be included in the skip list
## file_name = "enron_unique.csv"
## df_emails = pd.read_csv(file, skiprows=skip)

# Bloc 2
#
file_name = "enron_unique_1000"
file = file_path+file_name+'.csv'
df_emails = pd.read_csv(file)

print('Fichier de départ : ',file)

df_emails['cc'] = ''                        # pour identfication des personnes en copie
df_emails['NER_header'] = ''                # Spacy NER
df_emails['NER_body'] = ''                  # Spacy NER
df_emails['body_clean'] = ''                # body sans caracteres parasites
df_emails['body_dict'] = ''                 # body tokenisé
df_emails['summary_TFIDF'] = ''             # extractive summary by TF-IDF
df_emails['summary_TFIDF_sim'] = ''         # calcul de similarité par rapport à body_clean
df_emails['summary_spacy'] = ''             # extractive summary by Spacy
df_emails['summary_spacy_sim'] = ''         # calcul de similarité par rapport à body_clean
df_emails['summary_BART'] = ''              # abstractive summary by HugginFace/BART
df_emails['summary_BART_sim'] = ''          # calcul de similarité par rapport à body_clean
df_emails['summary_T5'] = ''                # abstractive summary by HugginFace/T5
df_emails['summary_T5_sim'] = ''            # calcul de similarité par rapport à body_clean
df_emails['best_sim'] = ''                  # meilleure taux de similarité par rapport à body_clean

df_emails.tail(10)
deb = 0
fin = df_emails.shape[0]
print(deb,fin)

file_name_out = "enron_output"
file_name_out = file_name_out+'_'+str(fin)
print(file_name)


Fichier de départ :  /content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_unique_1000.csv
0 1000
enron_unique_1000


Fonction - Sauvegarde vers CSV

In [14]:
# SAUVEGARDE en csv du résultat.
def svgd_csv_xls (file_path,file_name):

# to .CSV
  file_extension = ".csv"
  file = file_path+file_name+file_extension
  print("File : ",file)
  df_emails.to_csv(file, encoding='utf-8', index=False)
# df_emails[deb:fin].to_csv(file, encoding='utf-8', index=False)
  print("Svgde effectuée")
  return


# Data Cleaning

In [15]:
# création du champ body_clean
df_emails['body_clean'] = ''                # body sans caracteres parasites


1. DataCleaning



> 1a) Regex



In [16]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import re
import string

def data_clean (tokens):
# Suppression des caractères spéciaux ou inutiles \"',]
  r = re.compile(r"[\\\''\'\"\[\]]+")
  tok=r.sub('', tokens)
  r = re.compile(r"([\(\)]{1,}|,{2,})")
  tok2=r.sub(' ', tok)
  r = re.compile(r"((\.\s?,)+|(, ,)|(^,{1,}))")
  tok3=r.sub('', tok2)
  r = re.compile(r"\s{2,}")
  text=r.sub(' ', tok3)

# Rajout 08/08
  text = text.lower()
  text = re.sub('\[.*?\]', '', text)
  text = re.sub('https?://\S+|www\.\S+', '', text)
  text = re.sub('<.*?>+', '', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  text = re.sub('\n', '', text)
  text = re.sub('\w*\d\w*', '', text)

  return text
# -----------------------------------------------------------------------------
deb = 0
fin = df_emails.shape[0]

print("DataCleaning 1/2 en cours pour ",fin, "records.")
df_emails['body_clean'] = df_emails['body'].apply(lambda x : data_clean(x))
print("DataCleaning 1/2 terminé pour ",fin, "records.")


DataCleaning 1/2 en cours pour  1000 records.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


DataCleaning 1/2 terminé pour  1000 records.


> 1b) Suppression de ce qui precede le "subject" dans le ['body']

In [17]:
# suppression dans le corps du texte de tout ce qui precede la mention 'subject:" y compris le subject"
def data_clean_header_in_body(header,body):
  # Suppression de tout ce qui précède l'objet (inclus) dans le corps du mail
  # header = df_emails['header'][id]
  # body = df_emails['body_clean'][id]
  ref = 'Subject: '+str(header)+' '
  result = body.find(ref)
  if result != -1:
    result +=len(ref)
    texte = body[result:]
  else:
    texte = body
  # Traitement des mails avec header vide mais comportement Subject dans le body : on supprime tout ce qui précéde jusqu'à  subject
  ref = 'Subject: '
  result = body.find(ref)
  if result != -1:
    result +=len(ref)
    texte = body[result:]
  else:
    texte = body
  return texte
# -----------------------------------------------------------------------------
print("DataCleaning 2/2 en cours pour ",fin, "records.")
for id in range(deb,fin):
#  print('ID:',id)
  try:
    header = df_emails['header'][id]
    body = df_emails['body_clean'][id]
    df_emails['body_clean'][id] = data_clean_header_in_body(header,body)
#    print('ID:',id)
#    print(header,' // ',body)
  except:
    print('erreur retraitement body ID',id)
print("DataCleaning terminé pour ",fin, "records.")




DataCleaning 2/2 en cours pour  1000 records.
DataCleaning terminé pour  1000 records.


Sauvegarde au format csv

In [18]:

svgd_csv_xls (file_path,file_name_out)

File :  /content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_output_1000.csv
Svgde effectuée
